In [6]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb  9 16:48:46 2023

@author: oussama
"""

import sim
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import numpy as np


def split_image(image):
    image = np.array(image, dtype=np.uint8).reshape((256, 256, 3))
    image = np.flipud(image)
    height, width, channels = image.shape
    # Check that the image has even dimensions
    if height % 2 != 0 or width % 2 != 0:
        raise ValueError('Image dimensions must be even')

    # Split the image into four quadrants
    quadrant_height = height // 2
    quadrant_width = width // 2
    top_left = image[:quadrant_height, :quadrant_width]
    top_right = image[:quadrant_height, quadrant_width:]
    bottom_left = image[quadrant_height:, :quadrant_width]
    bottom_right = image[quadrant_height:, quadrant_width:]

    return top_left, top_right, bottom_left, bottom_right
def find_greenest_half(image):
    # Convert image data to NumPy array
    # image = np.array(image_data, dtype=np.uint8).reshape((256, 256, 3))
    
    # Reshape the array to the correct dimensions
    height, width, channels = image.shape
    
    # Flip the image vertically
    # image = cv2.flip(image, 0)
    
    # Check that the image has even dimensions
    if height % 2 != 0:
        raise ValueError('Image height must be even')
    
    # Split the image into two halves
    half_height = height // 2
    top_half = image[:half_height, :]
    bottom_half = image[half_height:, :]
    
    # Threshold the green channel of each half to create a binary image
    threshold_value = 80
    _, top_half_binary = cv2.threshold(top_half[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    _, bottom_half_binary = cv2.threshold(bottom_half[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    
    # Find the largest contour in each binary image
    contours = []
    for binary in (top_half_binary, bottom_half_binary):
        half_contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(half_contours) > 0:
            contour = max(half_contours, key=cv2.contourArea)
            contours.append(contour)
    
    # Calculate the percentage of green pixels in the largest contour of each half
    green_percentages = []
    for half, contour in zip((top_half, bottom_half), contours):
        green_mask = (half[:, :, 1] > half[:, :, 0]) & (half[:, :, 1] > half[:, :, 2])
        green_pixels = np.count_nonzero(green_mask & cv2.drawContours(np.zeros(half.shape[:2], dtype=np.uint8), [contour], -1, 255, -1))
        total_pixels = cv2.contourArea(contour)
        green_percent = green_pixels / total_pixels
        green_percentages.append(green_percent)
    
    # Return the half with the highest percentage of green pixels
    if green_percentages[0] > green_percentages[1]:
        return top_half
    else:
        return bottom_half

def find_greenest_quadrant(image):
    # Convert image data to NumPy array
    #image = np.array(image_data, dtype=np.uint8).reshape((256, 256, 3))
    
    # Reshape the array to the correct dimensions
    height, width, channels = image.shape

    
    # Flip the image vertically
    
    
    # Check that the image has even dimensions
    if height % 2 != 0 or width % 2 != 0:
        raise ValueError('Image dimensions must be even')
    
    # Split the image into four quadrants
    quadrant_height = height // 2
    quadrant_width = width // 2
    top_left = image[:quadrant_height, :quadrant_width]
    top_right = image[:quadrant_height, quadrant_width:]
    bottom_left = image[quadrant_height:, :quadrant_width]
    bottom_right = image[quadrant_height:, quadrant_width:]
    
    # Threshold the green channel of each quadrant to create a binary image
    threshold_value = 80
    _, top_left_binary = cv2.threshold(top_left[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    _, top_right_binary = cv2.threshold(top_right[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    _, bottom_left_binary = cv2.threshold(bottom_left[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    _, bottom_right_binary = cv2.threshold(bottom_right[:, :, 1], threshold_value, 255, cv2.THRESH_BINARY)
    
    # Find the largest contour in each binary image
    contours = []
    for binary in (top_left_binary, top_right_binary, bottom_left_binary, bottom_right_binary):
        quadrant_contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if len(quadrant_contours) > 0:
            contour = max(quadrant_contours, key=cv2.contourArea)
            contours.append(contour)
    
    # Calculate the percentage of green pixels in the largest contour of each quadrant
    green_percentages = []
    for quadrant, contour in zip((top_left, top_right, bottom_left, bottom_right), contours):
        green_mask = (quadrant[:, :, 1] > quadrant[:, :, 0]) & (quadrant[:, :, 1] > quadrant[:, :, 2])
        green_pixels = np.count_nonzero(green_mask & cv2.drawContours(np.zeros(quadrant.shape[:2], dtype=np.uint8), [contour], -1, 255, -1))
        total_pixels = cv2.contourArea(contour)
        green_percent = green_pixels / total_pixels
        green_percentages.append(green_percent)
    
    # Return the quadrant with the highest percentage of green pixels
    max_index = np.argmax(green_percentages)
    if max_index == 0:
        return top_left, 1
    elif max_index == 1:
        return top_right, 2
    elif max_index == 2:
        quadrant_num = 3
        return bottom_left, 3
    else:
        quadrant_num = 4
        return bottom_right, 4



print("program started")
sim.simxFinish(-1)  # close all open connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)
Vision_Data = []
if clientID != -1:  # check if client connection successful
    print('Connected to remote API server')
    res, objs = sim.simxGetObjects(
        clientID, sim.sim_handle_all, sim.simx_opmode_blocking)
    if res == sim.simx_return_ok:
        print('Number of objects', len(objs))
    else:
        print('RemoteAPI function call returned with error')
    result, vision_sensor_handle = sim.simxGetObjectHandle(
        clientID, "Vision_sensor", sim.simx_opmode_oneshot_wait)
    if result == sim.simx_return_ok:
        print("Got handle for vision")

        # start the simulation
        #  sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)
        # capture an image from vision sensor

        t0 = sim.simxGetLastCmdTime(clientID)
    else:
        print("Error getting handle for vision sensor")
else:
    print('Connection failed')
    sys.exit('Could not connect')





program started
Connected to remote API server
Number of objects 63
Got handle for vision


In [12]:
resultImg,resolution, image=sim.simxGetVisionSensorImage(clientID, vision_sensor_handle, 0, sim.simx_opmode_streaming)
resultImg,resolution, image=sim.simxGetVisionSensorImage(clientID, vision_sensor_handle, 0, sim.simx_opmode_buffer)
if resultImg == sim.simx_return_ok :
    print("Got the image")
else:
    print("Could not get the image array")
errorCode_target, target_handle = sim.simxGetObjectHandle(clientID,'./Mytarget', sim.simx_opmode_blocking)
pos_error, position = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_streaming)
pos_error, position = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_buffer)
if errorCode_target == sim.simx_return_ok :
    print("Got the drone target handle")
else:
    print("Could not get the drone target handle")
if pos_error == sim.simx_return_ok :
    print("Got the position of the target")
    print(position)
else:
    print("Could not get the position of the target")

Got the image
Got the drone target handle
Got the position of the target
[-0.7499999403953552, 1.5749999284744263, 0.5]


In [ ]:
simStep = 0.01
simEnd = 10

sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)

resultImg,resolution, image=sim.simxGetVisionSensorImage(clientID, vision_sensor_handle, 0, sim.simx_opmode_buffer)
pos_error, position = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_streaming)
while (sim.simxGetConnectionId(clientID) or sim.simxGetLastCmdTime(clientID)< simEnd):
    resultImg,resolution, image=sim.simxGetVisionSensorImage(clientID, vision_sensor_handle, 0, sim.simx_opmode_buffer)
    # Image Processing
    if resultImg == sim.simx_return_ok:
        image_data = np.array(image, dtype=np.uint8).reshape((256, 256, 3))
        greenest_quadrant, num= find_greenest_quadrant(image_data)
        pos_error, position = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_buffer)
        if pos_error == sim.simx_return_ok :
            if (num == 1):
                position[0] += 0.01
                position[1] += 0.01
                sim.simxSetObjectPosition(clientID, target_handle, -1, position,  sim.simx_opmode_oneshot) 
            elif ( num == 2):
                position[0] += 0.01
                position[1] -= 0.01
                sim.simxSetObjectPosition(clientID, target_handle, -1, position,  sim.simx_opmode_oneshot) 
            elif ( num == 3 ):
                position[0] += 0.01
                position[1] += 0.01
                sim.simxSetObjectPosition(clientID, target_handle, -1, position,  sim.simx_opmode_oneshot) 
            elif (num == 4 ):
                position[0] += 0.01
                position[1] -= 0.01
                sim.simxSetObjectPosition(clientID, target_handle, -1, position,  sim.simx_opmode_oneshot) 
        else :
            print("unvailable position of the object")
        # Display the image
        #cv2.imshow('Vision Sensor Image', image)
        #key = cv2.waitKey(1)

        # Check if the 'q' key was pressed
        #if key & 0xFF == ord('q'):
                #break

    # Sleep for the simulation time step
    time.sleep(simStep)

# Stop the simulation
#sim.simxStopSimulation(clientID, sim.simx_opmode_oneshot)

the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone movi

the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone moving
the drone movi

In [ ]:
top_left, top_right, bottom_left, bottom_right = split_image(image)
# Display the quadrants in separate windows
cv2.imshow('top left', top_left)
cv2.imshow('top right', top_right)
cv2.imshow('bottom left', bottom_left)
cv2.imshow('bottom right', bottom_right)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
image_data = np.array(image, dtype=np.uint8).reshape((256, 256, 3))
greenest_quadrant, num= find_greenest_quadrant(image_data)
print(num)
cv2.imshow('greenest quadrant', greenest_quadrant)
cv2.waitKey(0)
cv2.destroyAllWindows()

1
